In [ ]:
!pip install transformers datasets seqeval

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [ ]:
raw_datasets["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [ ]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [ ]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)


EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer.is_fast

True

In [ ]:
# tokenizer.save_pretrained('/content/drive/MyDrive/NER_Bert')

In [ ]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [ ]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [ ]:
tokenized_datasets['test'][0]

{'input_ids': [101,
  156,
  9244,
  10954,
  2069,
  118,
  147,
  12240,
  14962,
  25075,
  1942,
  149,
  21986,
  2428,
  3663,
  160,
  11607,
  117,
  24890,
  11607,
  1592,
  15969,
  156,
  19556,
  22861,
  6258,
  2036,
  18581,
  2271,
  12420,
  1942,
  119,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  2,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -100]}

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, return_tensors="tf"
)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<tf.Tensor: shape=(2, 12), dtype=int64, numpy=
array([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0,
        -100],
       [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100,
        -100]])>

In [ ]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


In [ ]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

In [ ]:
tf_train_dataset

<_PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>

In [ ]:
for batch in tf_train_dataset:
  print(batch)
  break

{'input_ids': <tf.Tensor: shape=(16, 57), dtype=int64, numpy=
array([[  101,  2123,  1489,  9784,  1765,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  101,  1130,  1901,  1106,  1606, 11438,  4196,   117,  1103,
        23195,  1209,  1145,  1138,  1106,  1129,  6228,   117,  1250,
         1218,  1107,   170,  1264,   117,  1129,  7246,  1106,  8132,
         2993,  1105,  1383, 18222,  2513,   119,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  101,   156, 2270

In [ ]:
for batch in tf_eval_dataset:
  print(batch)
  break

{'input_ids': <tf.Tensor: shape=(16, 48), dtype=int64, numpy=
array([[  101, 15531,  9741, 22441,  1942,   118,   149, 27514, 10954,
         9272,  9637,  1708,  3048, 18172,  2036,   157,  1592, 22441,
          152, 17145,  2069, 13020, 16972,  2101,   138, 26321,  9637,
        15969, 27451, 11780,  1708,  7118, 16647,  9565,  3663,   119,
          102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  101,   149, 11414,  2137, 11414,  1820,   118,  4775,   118,
         1476,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  101,  1537,  1890,  1155,   118,  1668,  1200,  5676, 14068,
         1261,  1300,  1111,  3383,  1113,  5286,  1112, 21854,  3222,
         8860,  1118,  112

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
id2label

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [ ]:
label2id

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [ ]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import create_optimizer
import tensorflow as tf

# Train in mixed-precision float16
# Comment this line out if you're using a GPU that will not benefit from this
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_epochs = 1
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

In [ ]:
optimizer

In [ ]:
# from transformers.keras_callbacks import PushToHubCallback

# callback = PushToHubCallback(output_dir="bert-finetuned-ner", tokenizer=tokenizer)

# model.fit(
#     tf_train_dataset,
#     validation_data=tf_eval_dataset,
#     # callbacks=[callback],
#     epochs=num_epochs,
# )

878/878 [==============================] - 167s 150ms/step - loss: 0.1901 - val_loss: 0.0707


In [ ]:
# model.save_pretrained('/content/drive/MyDrive/NER_Bert')


In [ ]:
# model.save_weights('/content/drive/MyDrive/NER_Bert')


In [ ]:
from transformers import TFAutoModelForTokenClassification
model = TFAutoModelForTokenClassification.from_pretrained('/content/drive/MyDrive/NER_Bert')

Some layers from the model checkpoint at /content/drive/MyDrive/NER_Bert were not used when initializing TFBertForTokenClassification: ['dropout_151']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at /content/drive/MyDrive/NER_Bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [ ]:
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [ ]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [ ]:
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [ ]:
predictions

['B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [ ]:
tf_train_dataset

<_PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>

In [ ]:
len(tf_train_dataset)

878

In [ ]:
tf_eval_dataset

<_PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>

In [ ]:
len(tf_eval_dataset)

204

In [ ]:
for batch in tf_eval_dataset:
  print(batch)
  break

{'input_ids': <tf.Tensor: shape=(16, 48), dtype=int64, numpy=
array([[  101, 15531,  9741, 22441,  1942,   118,   149, 27514, 10954,
         9272,  9637,  1708,  3048, 18172,  2036,   157,  1592, 22441,
          152, 17145,  2069, 13020, 16972,  2101,   138, 26321,  9637,
        15969, 27451, 11780,  1708,  7118, 16647,  9565,  3663,   119,
          102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  101,   149, 11414,  2137, 11414,  1820,   118,  4775,   118,
         1476,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  101,  1537,  1890,  1155,   118,  1668,  1200,  5676, 14068,
         1261,  1300,  1111,  3383,  1113,  5286,  1112, 21854,  3222,
         8860,  1118,  112

In [ ]:
for batch in tf_eval_dataset:
  print(batch)
  break

{'input_ids': <tf.Tensor: shape=(16, 48), dtype=int64, numpy=
array([[  101, 15531,  9741, 22441,  1942,   118,   149, 27514, 10954,
         9272,  9637,  1708,  3048, 18172,  2036,   157,  1592, 22441,
          152, 17145,  2069, 13020, 16972,  2101,   138, 26321,  9637,
        15969, 27451, 11780,  1708,  7118, 16647,  9565,  3663,   119,
          102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  101,   149, 11414,  2137, 11414,  1820,   118,  4775,   118,
         1476,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  101,  1537,  1890,  1155,   118,  1668,  1200,  5676, 14068,
         1261,  1300,  1111,  3383,  1113,  5286,  1112, 21854,  3222,
         8860,  1118,  112

In [ ]:
logits = model.predict_on_batch(batch)['logits']
logits

array([[[ 4.746  , -0.965  , -0.6777 , ..., -0.8057 , -0.1145 ,
         -0.2905 ],
        [ 6.848  , -0.9863 , -1.234  , ..., -1.203  , -0.4536 ,
         -0.5415 ],
        [ 6.543  , -1.348  , -0.3247 , ..., -0.7856 , -1.616  ,
          0.4424 ],
        ...,
        [ 0.3215 ,  0.53   ,  0.1543 , ..., -0.4553 , -0.07117,
         -0.1472 ],
        [ 1.939  , -0.7183 ,  0.4077 , ...,  0.07635, -0.785  ,
          0.4956 ],
        [ 1.462  , -0.5083 , -0.2869 , ..., -0.02351, -0.2155 ,
          0.09796]],

       [[ 3.734  , -0.837  , -0.975  , ..., -0.628  ,  0.436  ,
         -0.3157 ],
        [-0.6924 , -0.2247 , -1.826  , ...,  0.568  , -0.1643 ,
         -1.141  ],
        [-1.161  , -1.792  ,  0.277  , ...,  6.668  , -1.125  ,
         -0.3792 ],
        ...,
        [ 2.156  , -0.734  , -1.614  , ..., -0.3618 ,  0.2471 ,
         -1.212  ],
        [-0.791  , -0.3032 , -1.711  , ...,  1.093  , -0.3984 ,
         -1.173  ],
        [-0.8345 , -0.3613 , -1.683  , ...,  1.4

In [ ]:
logits.shape

(16, 48, 9)

In [ ]:
logits[0]

array([[ 4.746   , -0.965   , -0.6777  , -0.787   , -0.4543  , -1.571   ,
        -0.8057  , -0.1145  , -0.2905  ],
       [ 6.848   , -0.9863  , -1.234   , -0.645   , -0.565   , -1.486   ,
        -1.203   , -0.4536  , -0.5415  ],
       [ 6.543   , -1.348   , -0.3247  , -1.607   ,  0.0994  , -1.827   ,
        -0.7856  , -1.616   ,  0.4424  ],
       [ 6.523   , -1.302   , -0.423   , -1.5     , -0.00902 , -1.956   ,
        -0.824   , -1.443   ,  0.269   ],
       [ 6.555   , -1.629   , -0.583   , -1.532   , -0.0312  , -1.931   ,
        -0.757   , -1.336   ,  0.456   ],
       [ 6.164   , -0.5547  , -1.297   , -0.6934  , -1.265   , -1.404   ,
        -1.953   , -0.3894  , -0.3042  ],
       [ 0.9775  ,  2.365   , -1.336   ,  0.9463  , -2.03    ,  0.3535  ,
        -1.675   ,  1.867   , -1.114   ],
       [ 1.51    , -1.253   ,  1.613   , -2.29    , -0.1292  , -2.234   ,
         1.214   , -1.285   ,  3.059   ],
       [ 1.518   , -1.026   ,  2.004   , -2.377   ,  0.3345  , -1.78    

In [ ]:
logits[0].shape

(48, 9)

In [ ]:
labels = batch["labels"]
labels

<tf.Tensor: shape=(16, 48), dtype=int64, numpy=
array([[-100,    0,    0,    0,    0,    0,    3,    4,    4,    4,    4,
           4,    4,    4,    4,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100],
       [-100,    5,    6,    6,    6,    0,    0,    0,    0,    0, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100],
       [-100,    7,    8,    0,    0,    0,    0,    1,    2,    0,    0,
           0,    0,    0,    0,    0,    3,    0,    3,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0, -100, -100, -100, -100, -100

In [ ]:
import numpy as np
predictions = np.argmax(logits, axis=-1)

In [ ]:
predictions

array([[0, 0, 0, 0, 0, 0, 1, 8, 8, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 7, 1, 7, 7,
        7, 1, 0, 0],
       [0, 5, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 6, 6, 6, 6, 0,
        0, 0, 0, 0, 0, 0, 5, 5, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0,
        0, 5, 5, 5],
       [0, 7, 8, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 7, 7,
        7, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 3, 0,
        0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3,
        3, 3, 0, 0],
       [0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 3, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0,
        3, 3, 0, 0],
       [0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       

In [ ]:
predictions.shape

(16, 48)

In [ ]:
test_predictions = []
test_labels = []
for prediction, label in zip(predictions, labels):
  for predicted_idx, label_idx in zip(prediction, label):
    if label_idx == -100:
      continue
    test_predictions.append(label_names[predicted_idx])
    test_labels.append(label_names[label_idx])


In [ ]:
test_predictions

['O',
 'B-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
len(test_predictions)

51

In [ ]:
test_labels

['O',
 'B-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
test_labels

['O',
 'B-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
metric.compute(predictions=[test_predictions], references=[test_labels])

{'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
all_predictions = []
all_labels = []
for batch in tf_eval_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            all_predictions.append(label_names[predicted_idx])
            all_labels.append(label_names[label_idx])
metric.compute(predictions=[all_predictions], references=[all_labels])

{'LOC': {'precision': 0.9180503481521157,
  'recall': 0.9330430048992924,
  'f1': 0.9254859611231101,
  'number': 1837},
 'MISC': {'precision': 0.7160377358490566,
  'recall': 0.8232104121475055,
  'f1': 0.7658930373360242,
  'number': 922},
 'ORG': {'precision': 0.8583877995642701,
  'recall': 0.8814317673378076,
  'f1': 0.8697571743929359,
  'number': 1341},
 'PER': {'precision': 0.9369791666666667,
  'recall': 0.9766558089033659,
  'f1': 0.9564061669324827,
  'number': 1842},
 'overall_precision': 0.8762853470437018,
 'overall_recall': 0.9178727701110737,
 'overall_f1': 0.8965970738122636,
 'overall_accuracy': 0.9791899687996704}

In [ ]:
all_predictions

['O',
 'O',
 'O',
 'O',
 'O',
 'B-PER',
 'I-MISC',
 'I-MISC',
 'I-PER',
 'I-PER',
 'I-PER',
 'I-PER',
 'I-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'I-LOC',
 'I-LOC',
 'I-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-MISC',
 'I-MISC',
 'O',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'B-ORG',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'I-LOC',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '

In [ ]:
len(all_predictions)

67948

In [ ]:
len(all_labels)

67948

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/content/drive/MyDrive/NER_Bert"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")


Some layers from the model checkpoint at /content/drive/MyDrive/NER_Bert were not used when initializing TFBertForTokenClassification: ['dropout_151']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at /content/drive/MyDrive/NER_Bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


[{'entity_group': 'PER',
  'score': 0.9850526,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.67914295,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9874817,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
len("My name is Muhlis cm and I am from india. I am a python Developer working at Turbolab Technologies")

98

In [ ]:
token_classifier("My name is Muhlis cm and I am from india. I am a python Developer working at Turbolab Technologies")

[{'entity_group': 'PER',
  'score': 0.9125912,
  'word': 'Muhlis cm',
  'start': 11,
  'end': 20},
 {'entity_group': 'LOC',
  'score': 0.6270681,
  'word': 'india',
  'start': 35,
  'end': 40},
 {'entity_group': 'ORG',
  'score': 0.9431938,
  'word': 'Turbolab Technologies',
  'start': 77,
  'end': 98}]